In [13]:

%load_ext autoreload
%autoreload 2
import os
import dask
import panel
import pandas as pd
import matplotlib.pyplot as plt
import nisardev as nisar
import grimpfunc as grimp
import matplotlib.pyplot as plt
from dask.diagnostics import ProgressBar
import geopandas as gpd
ProgressBar().register()
dask.config.set(num_workers=2)  # Avoid problems with too many open connections at NSIDC
panel.extension() 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
env = dict(GDAL_HTTP_COOKIEFILE = os.path.expanduser('~/.grimp_download_cookiejar.txt'),
            GDAL_HTTP_COOKIEJAR = os.path.expanduser('~/.grimp_download_cookiejar.txt'))
os.environ.update(env)

import earthaccess
earthaccess.login() 

In [15]:
# For some environments the tool is unresponsive (i.e., search button doesn't work) - this can often be fixed by re-running this cell
myUrls = grimp.cmrUrls(mode='subsetter')
display(myUrls.initialSearch(product='NSIDC-0766'))
myCogs = myUrls.getCogs(replace='vv', removeTiff=True)

BokehModel(combine_events=True, render_bundle={'docs_json': {'9357add5-03ad-4b41-9857-2d339735f80d': {'version…

In [19]:
terminus_path = "/Users/sofiasuhinin/Desktop/GrIS_Calving/Jakobshavn-Isbrae-CalvingStatistics/data/MEaSUREs_0781/collated/JakbobshavnIsbraeTerminus.shp"
terminus = gpd.read_file(terminus_path)
terminus = terminus.to_crs("EPSG:3413")

minx, miny, maxx, maxy = terminus.total_bounds
margin = 15000

bbox = {"minx": minx-margin, "maxx": maxx+margin, "miny": miny-margin, "maxy": maxy + margin}
print("bbox:", bbox)

bbox: {'minx': np.float64(-205524.48300004005), 'maxx': np.float64(-164623.6277999878), 'miny': np.float64(-2293804.8136000633), 'maxy': np.float64(-2248855.862099886)}


In [ ]:
#oringial but for some reason velocity didnt intersect??
#warnings.filterwarnings("ignore", category=UserWarning, message="The argument 'infer_datetime_format' is deprecated*")  # Ignore warning about deprecated argument

# Define bounding box for subsetting the data
#bbox = {'minx': -250000, 'miny': -2250000, 'maxx': -150000, 'maxy': -2150000}

# Create a velocity series object with the COGs searched for above
#jiVelSeries = nisar.nisarVelSeries()
#jiVelSeries.readSeriesFromTiff(myCogs, url=True, readSpeed=False)

# Subset the velocity series to the bounding box
#jiVelSeries.subSetVel(bbox)

/var/folders/3r/pr708byx3wq5670k7_scfc240000gn/T/ipykernel_74886/3981051493.py:11: DeprecationWarning: 
subSetVel deprecated. Use subsetVel
  jiVelSeries.subSetVel(bbox)


In [20]:
#warnings.filterwarnings("ignore", category=UserWarning, message="The argument 'infer_datetime_format' is deprecated*")  # Ignore warning about deprecated argument

# Define bounding box for subsetting the data
#bbox = {'minx': -194000,'maxx': -170000,'miny': -2288000,'maxy': -2264000}

# Create a velocity series object with the COGs searched for above
jiVelSeries = nisar.nisarVelSeries()
jiVelSeries.readSeriesFromTiff(myCogs, url=True, readSpeed=False)

# Subset the velocity series to the bounding box
jiVelSeries.subSetVel(bbox)

/var/folders/3r/pr708byx3wq5670k7_scfc240000gn/T/ipykernel_8062/3843825878.py:11: DeprecationWarning: 
subSetVel deprecated. Use subsetVel
  jiVelSeries.subSetVel(bbox)


In [21]:
jiVelSeries.xr

<xarray.DataArray 'VelocityMap' (time: 463, band: 3, y: 13700, x: 7585)> Size: 577GB
dask.array<getitem, shape=(463, 3, 13700, 7585), dtype=float32, chunksize=(1, 1, 13700, 7585), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 61kB -6.59e+05 -6.588e+05 ... 8.576e+05 8.578e+05
  * y            (y) float64 110kB -6.392e+05 -6.394e+05 ... -3.379e+06
  * time         (time) datetime64[ns] 4kB 2015-01-06T12:00:00 ... 2024-12-14...
  * band         (band) <U2 24B 'vx' 'vy' 'vv'
    name         <U4 16B 'None'
    _FillValue   (band) float64 24B -2e+09 -2e+09 -1.0
    spatial_ref  int64 8B 0
    time1        (time) datetime64[ns] 4kB 2015-01-01 2015-01-13 ... 2024-12-09
    time2        (time) datetime64[ns] 4kB 2015-01-12 2015-01-24 ... 2024-12-20

In [22]:
output_dir ='../GrIS_Calving/MeASUREs_0766/'
output_file = os.path.join(output_dir, 'JakobshavnIsbraeVelocitySeriesNew.nc')

os.makedirs(output_dir, exist_ok=True)

jiVelSeries.toNetCDF(output_file)
    

[########################################] | 100% Completed | 427.14 s
[########################################] | 100% Completed | 427.24 s
[########################################] | 100% Completed | 427.35 s


'../GrIS_Calving/MeASUREs_0766/JakobshavnIsbraeVelocitySeriesNew.nc'